## Importando bibliotecas

In [ ]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from prophet import Prophet
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

## Puxando Dataframe

In [ ]:
%run Pre_Processamento.ipynb

In [ ]:
%store -r df_1

In [ ]:
df = df_1
df

## Preparação dos Dados

Nesta etapa, os dados foram divididos em duas versões distintas para testar e comparar os resultados do modelo sob diferentes condições:

Ajustando o nome da variavel items sold para "y" que será a Target do modelo Prophet.

In [ ]:
df = df.rename(columns={'items_sold': 'y'})

In [ ]:
df = df.rename(columns={'date': 'ds'})

Ordenando as datas de forma temporal e deixando no formato 'date' 

In [ ]:
df = df.sort_values(by='ds')

In [ ]:
df['ds'] = pd.to_datetime(df['ds']).dt.date

##### Remoção de Outliers (``df1``):

A partir do método IQR (Interquartile Range), outliers são identificados e removidos do dataset. Esta medida de dispersão estatística proporciona a amplitude interquartil, facilitando a detecção e filtragem de valores extremos. A aplicação desse método resultou em uma redução aproximada de 9% nos registros originais.

In [ ]:
q1 = df['y'].quantile(0.25)
q3 = df['y'].quantile(0.75)

iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

df1 = df[(df['y'] >= lower_bound) & (df['y'] <= upper_bound)]

In [ ]:
df.columns

##### Retirada da Black Friday (``df2``):

Nesta estratégia, semanas relacionadas à Black Friday e início de dezembro são excluídas, dado que tradicionalmente apresentam padrões atípicos de vendas.

In [ ]:
df2 = df.drop(df[df['semana_do_ano'].isin([46, 47, 48, 49])].index)

>Posteriormente, os dados foram divididos em conjuntos de treino e teste utilizando uma proporção de 70% para treino e 30% para teste.

In [ ]:
train_df1, test_df1 = train_test_split(df1, test_size=0.3, shuffle=False)

In [ ]:
train_df2, test_df2 = train_test_split(df2, test_size=0.3, shuffle=False)

## Aplicando o Modelo Prophet

### Treinamento sem Outliers

Treinamento com os conjuntos de treino

In [ ]:
model = Prophet(daily_seasonality=True, interval_width=0.95)
model.fit(train_df1)

In [ ]:
future = model.make_future_dataframe(periods=len(test_df1),freq="H", include_history=False)
forecast = model.predict(future)

In [ ]:
merged = forecast.set_index('ds')[['yhat']].join(test_df1.set_index('ds'))

In [ ]:
merged = merged.dropna(subset=['y', 'yhat'])

In [ ]:
import numpy as np

y_true = merged['y']
y_pred = merged['yhat']

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
mse2 = mean_squared_error(y_true, y_pred)

print(f"Erro Quadrático Médio (MSE): {mse2}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")
print(f"Erro Médio Absoluto (MAE): {mae}")

Os resultados obtidos são:

- Erro Médio Absoluto (MAE): 1.2980862188511293
- Erro Quadrático Médio (MSE): 2.804723476895441
- Raiz do Erro Quadrático Médio (RMSE): 1.674730867003842

| Métrica                               | Valor                  |
|---------------------------------------|------------------------|
| Erro Médio Absoluto (MAE)             | 1.2980862188511293    |
| Erro Quadrático Médio (MSE)           | 2.804723476895441      |
| Raiz do Erro Quadrático Médio (RMSE)  | 1.674730867003842      |

### Treinamento sem Semanas da Black Friday

Nesta fase, o modelo é treinado com os dados onde semanas específicas foram excluídas.

In [ ]:
model = Prophet(daily_seasonality=True, interval_width=0.95)
model.fit(train_df2)

In [ ]:
future = model.make_future_dataframe(periods=len(test_df2),freq="H", include_history=False)
forecast = model.predict(future)

In [ ]:
merged = forecast.set_index('ds')[['yhat']].join(test_df2.set_index('ds'))

In [ ]:
merged = merged.dropna(subset=['y', 'yhat'])

In [ ]:
import numpy as np

y_true = merged['y']
y_pred = merged['yhat']

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
mse2 = mean_squared_error(y_true, y_pred)

print(f"Erro Quadrático Médio (MSE): {mse2}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")
print(f"Erro Médio Absoluto (MAE): {mae}")

| Métrica                               | Valor                  |
|---------------------------------------|------------------------|
| Erro Médio Absoluto (MAE)             | 2.4319854207131555   |
| Erro Quadrático Médio (MSE)           | 20.95493097722733     |
| Raiz do Erro Quadrático Médio (RMSE)  | 4.577655620208595     |

## Discussão dos Resultados

Ao analisar os resultados, notamos as seguintes observações:

**Retirada de Outliers**: A estratégia de manusear outliers, eliminando-os, resultou em erro reduzido, evidenciando uma melhoria significativa na precisão das previsões do modelo. Isso pode ser interpretado como uma indicação de que, ao se abster de considerar os outliers, o modelo foca em tendências mais centrais dos dados. Entretanto, esta abordagem pode comprometer a capacidade do modelo de generalizar para casos mais extremos, uma vez que informações críticas podem ter sido descartadas.

**Retirada da Blach-Friday**: Diferentemente, a exclusão de dados relativos às semanas de Black Friday levou a um aumento nos erros de previsão, sinalizando uma perda de precisão. Surpreendentemente, o Erro Quadrático Médio (MSE) revelou-se substancialmente elevado, indicando que o modelo conseguiu assimilar uma amplitude maior da variabilidade intrínseca aos dados. Este fenômeno sugere que, apesar do modelo manifestar imprecisões maiores em certos pontos, sua capacidade de ajuste a diversas condições é notavelmente superior, possivelmente devido à inclusão de uma gama mais vasta de informações. Este ajuste mais amplo poderia ser interpretado como um reflexo de sua habilidade em processar e ajustar-se a uma diversidade maior de padrões e tendências presentes nos dados, oferecendo assim um retrato mais holístico e inclusivo das dinâmicas subjacentes. Isso pode ser crucial quando as previsões precisam abranger uma variedade mais extensa de cenários e condições.

**Justificativa da ausência do R²**: O coeficiente de determinação, r², comumente usado em modelos de regressão linear para avaliar a proporção da variância explicada pelo modelo, não é coletado no modelo Prophet devido às suas peculiaridades. O Prophet é projetado especificamente para séries temporais, focando em componentes de tendência e sazonalidade e em captar efeitos de eventos especiais e feriados, diferindo substancialmente dos modelos de regressão tradicionais onde o R² é aplicável. Este modelo prioriza a geração de previsões ponto a ponto e intervalos de confiança, com ênfase na adaptação a padrões não-lineares e anomalias, tornando outras métricas, como o Erro Médio Absoluto (MAE) e o Erro Quadrático Médio (MSE), mais pertinentes para avaliar sua precisão e adequação aos dados. Portanto, a ausência de R² no Prophet é justificada pela utilização de métricas de erro mais alinhadas aos objetivos de modelagem de séries temporais.

**Justificativa da ausência de ferramentas de otimização de hiperparâmetros (gridsearch e randomsearch)**: A não implementação de ferramentas de otimização, como RandomSearch e GridSearch, neste notebook decorre das limitações inerentes ao modelo Prophet. Este modelo, especificamente desenvolvido para análises de séries temporais, não possui suporte nem requer a configuração de hiperparâmetros tradicionais, que seriam otimizados através dessas técnicas de busca. O Prophet é construído com o objetivo de oferecer uma solução robusta e eficiente com configurações padrão, minimizando a necessidade de ajustes manuais e otimizações extensivas. Sua concepção prioriza a praticidade e a aplicabilidade, permitindo análises de alta qualidade sem a necessidade de tunning extenso de parâmetros, o que elimina a aplicabilidade de métodos de otimização como RandomSearch e GridSearch no contexto deste modelo.

# Considerações Finais

Após uma análise detalhada dos modelos em consideração (Random Forest, XGBoost e Prophet), chegamos à conclusão de que, em relação aos demais, o desempenho do modelo Prophet não obteve as melhores métricas de avaliação estabelecidos para este projeto. Devido a essas considerações, optamos por não escolher o Prophet como o modelo final.